In [21]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,extract_data_into_df,create_etl_watermark_table,insert_data_in_batches
from lookups import FileType
from prehook import return_csv_list,create_staging_tables
import prehook

db_session = create_connection()

In [22]:

stg_users_dataframe = pd.DataFrame({
     "age": [25, 30, 35],
     "gender": ['Male', 'Female', 'Male'],
     "fav_music_genre": ['Pop', 'Rock', 'Hip-Hop'],
     "music_time_slot": ['Morning', 'Evening', 'Afternoon'],
     "music_lis_frequency": ['Daily', 'Weekly', 'Monthly'],
     "music_satisfaction": ['High', 'Medium', 'Low']
 })

stg_songs_dataframe = pd.DataFrame({
     "track_id": [1, 2, 3],
     "track_name": ["Song1", "Song2", "Song3"],
     "track_artist": ["Artist1", "Artist2", "Artist3"],
     "track_popularity": [80, 90, 70],
     "track_album_id": [101, 102, 103],
     "track_album_name": ["Album1", "Album2", "Album3"],
     "track_album_release_date": ["2022-01-01", "2022-02-01", "2022-03-01"],
     "playlist_name": ["Playlist1", "Playlist2", "Playlist3"],
     "playlist_id": [201, 202, 203],
     "playlist_genre": ["Genre1", "Genre2", "Genre3"],
     "energy": [0.8, 0.7, 0.6],
     "key": [1, 2, 3],
     "loudness": [-5.0, -6.0, -7.0],
     "mode": [0, 1, 0],
     "speechiness": [0.1, 0.2, 0.3],
     "valence": [0.7, 0.6, 0.5],
     "tempo": [120, 130, 140],
     "duration_ms": [240000, 250000, 260000]
 })


In [23]:

stg_users_table_name = "stg_kaggle_spotify_users"
stg_users_schema_name = 'musicschema' 
stg_users_create_statement = return_create_statement_from_dataframe(
stg_users_dataframe, stg_users_schema_name, stg_users_table_name)

query = insert_statements_from_dataframe(stg_users_dataframe,stg_users_schema_name ,stg_users_table_name)

for statement in query:
    execute_query(db_session, statement)
    print(statement)
    
stg_songs_table_name = 'stg_kaggle_spotify_songs'
stg_songs_schema_name = 'musicschema'  
stg_songs_create_statement = return_create_statement_from_dataframe(
stg_songs_dataframe, stg_songs_schema_name, stg_songs_table_name)

execute_query(conn= db_session, query= stg_users_create_statement)

execute_query(conn= db_session, query= stg_songs_create_statement)

INSERT INTO musicschema.stg_kaggle_spotify_users (age, gender, fav_music_genre, music_time_slot, music_lis_frequency, music_satisfaction) VALUES (25, 'Male', 'Pop', 'Morning', 'Daily', 'High');
INSERT INTO musicschema.stg_kaggle_spotify_users (age, gender, fav_music_genre, music_time_slot, music_lis_frequency, music_satisfaction) VALUES (30, 'Female', 'Rock', 'Evening', 'Weekly', 'Medium');
INSERT INTO musicschema.stg_kaggle_spotify_users (age, gender, fav_music_genre, music_time_slot, music_lis_frequency, music_satisfaction) VALUES (35, 'Male', 'Hip-Hop', 'Afternoon', 'Monthly', 'Low');


In [24]:
from enum import Enum

class FileType(Enum):
    CSV = 1

datatype = FileType.CSV
datapath = "C:\\dataproject\\songs_dataset.csv"
df = extract_data_into_df(datatype,datapath)
df
#datapath2 = "C:\\dataproject\\users_dataset.csv"


In [25]:
# Create Dimension Tables
dim_users_data = stg_users_dataframe[["age", "gender", "fav_music_genre", "music_time_slot", "music_lis_frequency", "music_satisfaction"]]
dim_songs_data = stg_songs_dataframe[["track_id", "track_name", "track_artist", "playlist_genre"]]
dim_genre_data = stg_songs_dataframe[["playlist_genre"]]

dim_users_table_name = "Dim_users"
dim_songs_table_name = "Dim_songs"
dim_genre_table_name = "Dim_genre"

schema = 'musicschema'

# Execute create statements for dimension tables
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    dim_users_data, schema, dim_users_table_name))

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    dim_songs_data, schema, dim_songs_table_name))

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    dim_genre_data, schema, dim_genre_table_name))


In [26]:
# Create Fact Tables
fact_user_genre_data = stg_users_dataframe[["age", "gender", "fav_music_genre"]]
fact_user_genre_data["total_users"] = 1
fact_user_genre_data = fact_user_genre_data.groupby(["age", "gender", "fav_music_genre"]).sum().reset_index()

fact_song_genre_data = stg_songs_dataframe[["track_name", "track_artist", "playlist_genre"]]
fact_song_genre_data["total_songs"] = 1
fact_song_genre_data = fact_song_genre_data.groupby(["track_name", "track_artist", "playlist_genre"]).sum().reset_index()

fact_user_genre_table_name = "Fact_user_genre"
fact_song_genre_table_name = "Fact_song_genre"

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    fact_user_genre_data, schema, fact_user_genre_table_name))

execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    fact_song_genre_data, schema, fact_song_genre_table_name))



C:\Users\Pamela Bou Khalil\AppData\Local\Temp\ipykernel_24964\3298915897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_user_genre_data["total_users"] = 1
C:\Users\Pamela Bou Khalil\AppData\Local\Temp\ipykernel_24964\3298915897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_song_genre_data["total_songs"] = 1


In [27]:
# 1. Total songs per genre
total_songs_per_genre_data = stg_songs_dataframe.groupby("playlist_genre").size().reset_index(name="total_songs")
total_songs_per_genre_table_name = "Fact_total_songs_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    total_songs_per_genre_data, schema, total_songs_per_genre_table_name))

# 2. Total users per genre
total_users_per_genre_data = stg_users_dataframe.groupby("fav_music_genre").size().reset_index(name="total_users")
total_users_per_genre_table_name = "Fact_total_users_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    total_users_per_genre_data, schema, total_users_per_genre_table_name))

# 3. Average listen time per genre (Assuming you have a "duration_ms" column in your SONGS table)
average_listen_time_per_genre_data = stg_songs_dataframe.groupby("playlist_genre")["duration_ms"].mean().reset_index(name="average_listen_time")
average_listen_time_per_genre_table_name = "Fact_average_listen_time_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    average_listen_time_per_genre_data, schema, average_listen_time_per_genre_table_name))


# 5. Age distribution per genre
age_distribution_per_genre_data = stg_users_dataframe.groupby(["fav_music_genre", "age"]).size().reset_index(name="count")
age_distribution_per_genre_table_name = "Fact_age_distribution_per_genre"
execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    age_distribution_per_genre_data, schema, age_distribution_per_genre_table_name))


In [28]:
# Aggregate Tables
agg_genre_data = stg_songs_dataframe[["playlist_genre"]]
agg_genre_data["total_songs"] = 1
agg_genre_data = agg_genre_data.groupby("playlist_genre").sum().reset_index()

agg_dim_genre_data = stg_songs_dataframe[["playlist_genre"]]
agg_dim_genre_data = agg_dim_genre_data.drop_duplicates()

agg_total_songs_per_genre_table_name = "agg_total_songs_per_genre"
agg_dim_genre_table_name = "agg_dim_genre"


execute_query(conn=db_session, query=return_create_statement_from_dataframe(
    agg_genre_data, schema, agg_total_songs_per_genre_table_name))



C:\Users\Pamela Bou Khalil\AppData\Local\Temp\ipykernel_24964\3565597171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_genre_data["total_songs"] = 1


In [29]:
# import prehook  

# csv_list = prehook.return_csv_list()
# staging_table_name = "songs"  # Replace with your desired staging table name

# # Test the create_staging_tables function
# prehook.create_staging_tables(csv_list, staging_table_name)


In [30]:
watermark_table_sql_file ="C:\Data_project\SQL_Commands\V2_prehook_Create_ETL_watermark_table.sql"
data = pd.read_csv("C:\dataproject\songs_dataset.csv")
batch_size = 1000

create_etl_watermark_table(db_session, watermark_table_sql_file)

# Insert data in batches and record the ETL watermark timestamp
insert_data_in_batches(data, stg_songs_schema_name, stg_songs_table_name, batch_size, 'etl_watermark')

if db_session:
    db_session.close()

ETL watermark table created successfully.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
ETL watermark timestamp recorded.
Data inserted successfully in batches.
